In [704]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [705]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "PESEL",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         None, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [706]:
employees_df

,id,imię,nazwisko,PESEL,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca
0,1,Jadwiga,Staszel,66120484281,Wrocław,ul. Towarowa 13/9,48348263837,jadwiga.staszel@kuba.pl,2014-01-01,None,menadżer,73.7
1,2,Andrzej,Jurkiewicz,86112740656,Lądek-Zdrój,ul. Szczecińska 66/27,48519986403,andrzej.jurkiewicz@kuba.pl,2014-01-01,None,sprzedawca,56.9
2,3,Joanna,Konik,70112623341,Wrocław,ul. Rodzinna 16/51,48889526323,joanna.konik@kuba.pl,2014-01-01,None,sprzedawca,52.4
3,4,Maria,Czupryńska,72120275149,Białystok,ul. Begoniowa 36/33,48153944080,maria.czuprynska@kuba.pl,2014-01-01,2017-04-13,mechanik,54.1
11,5,Andżelika,Szmołda,67062013065,Wrocław,ul. Wacława Gąsiorowskiego 62/42,48561528282,andzelika.szmolda@kuba.pl,2014-01-01,None,informatyk,49.8
4,6,Artem,Kierzkowski,66071135195,Wrocław,ul. Zawiślańska 50/18,48638220114,artem.kierzkowski@kuba.pl,2014-08-10,None,mechanik,57.1
5,7,Aleksander,Rusin,90052680611,Wrocław,ul. Osiedle Kasztelańskie 84,48612897996,aleksander.rusin@kuba.pl,2014-09-10,None,mechanik,54.5
6,8,Illia,Włosek,61051438192,Opole,ul. Konwaliowa 73,48113837011,illia.wlosek@kuba.pl,2014-12-27,2017-02-23,mechanik,55.1
7,9,Janusz,Bednarski,95081822571,Wrocław,ul. Polna 17/45,48037892622,janusz.bednarski@kuba.pl,2015-02-15,None,pomocnik mechanika,35.2
8,10,Patrycja,Ivanova,73021581982,Wrocław,ul. Kolaniki 12/4,48041205472,patrycja.ivanova@kuba.pl,2016-11-15,None,diagnosta samochodowy,49.4


In [707]:
employees_df.to_csv("outputs/pracownicy.csv", index=False)

# Transakcje

In [708]:
no_transactions = np.random.poisson(2000)
begining_buy = 20

dates = random_transaction_dates(begining_date, end_date, no_transactions)

In [709]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [710]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)
year_ceil = [int(date[:4])-19 for date in dates[:20]]

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()
not_fixed_our_cars = our_cars[:]

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(5*max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
            jp += 1
            year_ceil.append(int(dates[i][:4])-19)
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if len(not_fixed_our_cars) >= 1 and np.random.rand() <= weights[1]: #fixing car we currently have
           client_ids[i] = 0
           car_ids[i] = np.random.choice(not_fixed_our_cars)
           not_fixed_our_cars.remove(car_ids[i])
        elif np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

In [711]:
len(year_ceil), len(pd.unique(client_ids))

(1464, 1465)

In [712]:
year_ceil

[1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,

In [713]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [714]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [715]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [716]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [717]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [718]:
year_floor = 1960
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])

menager = employees_df.iloc[0]
if int(menager["PESEL"][-2]) % 2 == 1:
    menager_sex = "M"
else:
    menager_sex = "K"
clients_df.loc[0] = [0, menager["imię"], menager["nazwisko"], menager_sex, menager["PESEL"], menager["miasto"], menager["adres"], False, menager["telefon"], menager["email"]]

unique_clients = np.unique(client_ids)
unique_clients = unique_clients[unique_clients != 0]
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'M'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'K'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
                             
clients_df.astype({"PESEL": str})

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Jadwiga,Staszel,K,66120484281,Wrocław,ul. Towarowa 13/9,False,48348263837,jadwiga.staszel@kuba.pl
1,1,Joanna,Baum,K,61041377218,Wrocław,ul. Tulipanów 18/17,False,48520692950,joabau61@gmail.com
2,2,Janina,Hrynkiewicz,K,95042490393,Żory,ul. Sadowa 50/22,False,48610118337,bl78k62w5u1471@opayq.com
3,3,Mirosław,Góral,M,85111610319,Wrocław,ul. Hetmańska 73/25,False,48772614287,m.goral1322@gmail.com
4,4,Kinga,Chmara,K,61020717774,Miechów,ul. Grodzisko 22/14,False,48495623841,chkinga@wp.pl
...,...,...,...,...,...,...,...,...,...,...
1460,1460,Marek,Polaszek,M,76022345713,Wrocław,ul. Senatorska 32/46,False,48804475119,vp5838oh38851g@opayq.com
1461,1461,Sławomir,Kloska,M,62062704531,Wrocław,ul. Promienna 86,False,48779699727,kloska622@outlook.com
1462,1462,Alan,Jędrzejak,M,83101585931,Wrocław,ul. Płocka 7/12,False,48977213298,a_je1983@wp.pl
1463,1463,Renata,Sikora,K,78030395678,Wrocław,ul. Melchiora Wierzbickiego 34,False,48204683614,r.sikor@gmail.com


In [719]:
clients_with_lc = clients_df[clients_df["karta_lojalnościowa"]]["id"].values

loyalty_card_dates = [() for i in range(len(clients_with_lc))]
for (i, client) in enumerate(clients_with_lc):
    loyalty_card_dates[i] = client, dates[np.where(client_ids == client)[0][2]]


In [720]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Jadwiga,Staszel,K,66120484281,Wrocław,ul. Towarowa 13/9,False,48348263837,jadwiga.staszel@kuba.pl
1,1,Joanna,Baum,K,61041377218,Wrocław,ul. Tulipanów 18/17,False,48520692950,joabau61@gmail.com
2,2,Janina,Hrynkiewicz,K,95042490393,Żory,ul. Sadowa 50/22,False,48610118337,bl78k62w5u1471@opayq.com
3,3,Mirosław,Góral,M,85111610319,Wrocław,ul. Hetmańska 73/25,False,48772614287,m.goral1322@gmail.com
4,4,Kinga,Chmara,K,61020717774,Miechów,ul. Grodzisko 22/14,False,48495623841,chkinga@wp.pl
...,...,...,...,...,...,...,...,...,...,...
1460,1460,Marek,Polaszek,M,76022345713,Wrocław,ul. Senatorska 32/46,False,48804475119,vp5838oh38851g@opayq.com
1461,1461,Sławomir,Kloska,M,62062704531,Wrocław,ul. Promienna 86,False,48779699727,kloska622@outlook.com
1462,1462,Alan,Jędrzejak,M,83101585931,Wrocław,ul. Płocka 7/12,False,48977213298,a_je1983@wp.pl
1463,1463,Renata,Sikora,K,78030395678,Wrocław,ul. Melchiora Wierzbickiego 34,False,48204683614,r.sikor@gmail.com


In [721]:
clients_df.to_csv("outputs/klienci.csv", index=False)

In [722]:
np.sum(clients_df['karta_lojalnościowa'])

77

In [723]:
np.sum(pd.isna(clients_df["PESEL"]))

45

# auta

In [724]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [725]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [726]:
df_samochody = create_car_table(max(car_ids))

In [727]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Volvo,V60,4,2012,Sedan,False,biały,Benzyna,4.5,288,Automatyczna,5,2,False
1,2,Audi,A4,4,2008,Sedan,True,niebieski,Diesel,2.8,177,Automatyczna,3,5,False
2,3,Kia,Ceed / cee'd,4,2009,Van,False,czarny,Benzyna,2.1,140,Manualna,5,2,False
3,4,Mazda,CX-30,4,2015,SUV,True,czerwony,Elektryczny/Benzyna,1.7,116,Manualna,5,5,False
4,5,Hyundai,Tucson,4,2010,Van,False,szary,Diesel,2.0,132,Manualna,3,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,942,Mercedes-Benz,Vito,4,2012,Sedan,False,srebrny,Diesel,2.5,163,Manualna,5,5,True
942,943,MINI,Cooper S Countryman,4,2005,Kombi,False,szary,Benzyna,2.8,190,Manualna,5,5,True
943,944,Volkswagen,up!,4,2003,Kombi,False,szary,Benzyna,0.7,60,Manualna,3,5,True
944,945,Toyota,Yaris,4,2010,Sedan,True,czerwony,Benzyna,1.1,72,Manualna,5,5,False


In [728]:
df_samochody.to_csv("outputs/samochody.csv", index=False)

In [729]:
pd.unique(df_samochody["skrzynia_biegów"])

array(['Automatyczna', 'Manualna', 'Półautomatyczna'], dtype=object)

# Usługi

In [730]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksięgowania'] = transaction_dates_service

# Zakup samochodów

In [731]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod) + 1) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [732]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksięgowania'] = transaction_dates_buy

In [733]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksięgowania'] = service_date_fix['data_zaksięgowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksięgowania'] = buy_date_fix['data_zaksięgowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\olkar\AppData\Local\Temp\ipykernel_26044\3526569416.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2005' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [734]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

In [735]:
df_buy

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,1,1,2,2014-01-03,167300.0
1,2,2,2,2,2014-01-05,46000.0
2,3,3,3,2,2014-01-07,56700.0
3,4,4,4,2,2014-01-08,22700.0
4,5,5,5,2,2014-01-09,53100.0
...,...,...,...,...,...,...
938,939,942,717,2,2017-12-28,78600.0
939,940,943,1460,2,2017-12-28,47200.0
940,941,944,1461,2,2017-12-28,17500.0
941,942,945,1462,3,2017-12-29,20100.0


In [736]:
df_buy.to_csv("outputs/zakup_samochodu.csv", index=False)

# Sprzedaż samochodów

In [737]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksięgowania'] = transaction_dates_sell

In [738]:
loyalty_card_customer_ids = [el[0] for el in loyalty_card_dates]
loyalty_card_dates_sep = [el[1] for el in loyalty_card_dates]

In [739]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

for i in range(len(sell_ids)):
    customer_id = df_sell.loc()[i]['id_klienta']
    if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:
        date_index = loyalty_card_customer_ids.index(customer_id)
        if loyalty_card_dates_sep[date_index] < df_sell.loc()[i]['data_zaksięgowania']:
            sell_prices[i] *= 0.97
            #print(i)

sell_installments = []
for i in range(len(sell_ids)):
    if sell_prices[i] > 80000:
        inst_prob = [0.1, 0.2, 0.3, 0.4]
    else:
        inst_prob = [0.4, 0.3, 0.2, 0.1]
    installments = rng.choice([1, 6, 12, 18], p=inst_prob)
    if installments == 6:
        sell_prices[i] *= 1.02
    elif installments == 12:
        sell_prices[i] *= 1.04
    elif installments == 18:
        sell_prices[i] *= 1.06
    sell_installments.append(installments)
#print(sell_installments)

df_sell['cena'] = sell_prices
df_sell['liczba_rat'] = sell_installments
df_sell = df_sell[['id', 'id_samochodu', 'id_klienta', 'id_pracownika', 'cena', 'liczba_rat', 'data_zaksięgowania']]

C:\Users\olkar\AppData\Local\Temp\ipykernel_26044\1822833189.py:8: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:


In [740]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,cena,liczba_rat,data_zaksięgowania
0,1,11,21,3,36108.0,6,2014-01-21
1,2,19,22,2,22984.0,12,2014-01-22
2,3,16,23,3,43100.0,1,2014-01-22
3,4,4,24,3,32946.0,6,2014-01-24
4,5,3,25,2,76500.0,6,2014-01-24
...,...,...,...,...,...,...,...
931,932,936,1455,3,30000.0,1,2017-12-24
932,933,934,1458,3,75790.0,18,2017-12-27
933,934,937,1459,2,58038.0,6,2017-12-27
934,935,941,1463,2,50388.0,6,2017-12-30


In [741]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

0

In [742]:
df_sell.to_csv("outputs/sprzedaż_samochodu.csv", index=False)

# Użyte części

In [743]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(32, array([40, 40], dtype=int64)), (48, array([30], dtype=int64)), (47, array([39], dtype=int64)), (52, array([22], dtype=int64)), (60, array([16], dtype=int64)), (64, array([29, 16], dtype=int64)), (66, array([59], dtype=int64)), (70, array([39, 33], dtype=int64)), (73, array([13], dtype=int64)), (96, array([39, 44], dtype=int64)), (109, array([63,  5], dtype=int64)), (110, array([44], dtype=int64)), (115, array([50], dtype=int64)), (118, array([32], dtype=int64)), (119, array([20], dtype=int64)), (120, array([36], dtype=int64)), (127, array([4], dtype=int64)), (136, array([52, 32], dtype=int64)), (140, array([55], dtype=int64)), (146, array([49], dtype=int64)), (147, array([23], dtype=int64)), (150, array([49], dtype=int64)), (155, array([28], dtype=int64)), (172, array([12], dtype=int64)), (171, array([43], dtype=int64)), (176, array([22], dtype=int64)), (181, array([47, 24, 59], dtype=int64)), (190, array([26], dtype=int64)), (200, array([ 7, 48], dtype=int64)), (204, array([10],

In [744]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [745]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Rozrząd,,2500,1,1
1,Rozrząd,,2500,1,1
2,Dywan,SEAT Leon,18,2,2
3,Silnik,SEAT Leon 2.3,4542,3,1
4,Światło wewnętrzne,Renault Megane,10,4,1
...,...,...,...,...,...
218,Zagłówek,Volkswagen up!,9,150,2
219,Chłodnica,SEAT Altea XL,589,151,1
220,Drzwi prawe przednie,SEAT Altea XL biały,413,151,1
221,Zbiornik paliwa,Kia Ceed / cee'd,399,152,1


# Wyposażenie

In [746]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [747]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Rozrząd,,2500.0
2,2,Dywan,SEAT Leon,18.0
3,3,Silnik,SEAT Leon 2.3,4542.0
4,4,Światło wewnętrzne,Renault Megane,10.0
5,5,Głośnik,Toyota Yaris,78.0
...,...,...,...,...
218,166,Zagłówek,Volkswagen up!,9.0
219,167,Chłodnica,SEAT Altea XL,589.0
220,168,Drzwi prawe przednie,SEAT Altea XL biały,413.0
221,169,Zbiornik paliwa,Kia Ceed / cee'd,399.0


# Usługi - uzupełnienie

In [748]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

df_services['cena'] = np.round(service_prices*rng.uniform(1.1, 1.4), -1).astype(int)

In [749]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,32,63,4,2014-02-18,6570
1,2,48,91,7,2014-03-09,50
2,3,47,0,7,2014-03-12,5970
3,4,52,101,8,2014-03-20,10
4,5,60,114,8,2014-03-28,100
...,...,...,...,...,...,...
147,148,909,1376,8,2017-11-02,3280
148,149,893,0,7,2017-11-14,3750
149,150,926,213,8,2017-11-28,20
150,151,30,1033,7,2017-12-07,1320


In [750]:
df_services.to_csv("outputs/usługi.csv", index=False)

# Użyte części - uzupełnienie

In [751]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość']]

In [752]:
used_parts_df

,id,id_usługi,id_części,ilość
0,1,1,1,1
1,2,1,1,1
2,3,2,2,2
3,4,3,3,1
4,5,4,4,1
...,...,...,...,...
218,219,150,166,2
219,220,151,167,1
220,221,151,168,1
221,222,152,169,1


In [753]:
used_parts_df.to_csv("outputs/użyte_części.csv", index=False)

# Zakup części

In [754]:
def get_service_date(part_id):
    
    index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
    service_date = df_services[df_services['id'] == index_used_part]['data_zaksięgowania']
    return service_date
    

buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'ilość'])

for i in range(len(buy_parts_quantity)):
    part_id = buy_parts_quantity['id_części'][i]
    quantity = buy_parts_quantity['ilość'][i]
    price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
    if price > 1000:
        quantity += rng.random() < 0.5
    elif price > 500:
        quantity += rng.choice([0, 1, 2], p=[0.2, 0.5, 0.3])
    else:
        quantity += rng.binomial(25, 0.15)
    total_cost = np.floor(quantity * price * rng.uniform(0.75, 0.95))[0]
    service_date = get_service_date(part_id)
    
    row = [int(i+1), int(part_id), total_cost, int(quantity)]
    parts_purchase_pd.loc[i] = row


In [755]:
parts_purchase_pd

,id,id_części,koszt,ilość
0,1.0,1.0,16456.0,7.0
1,2.0,2.0,114.0,8.0
2,3.0,3.0,8471.0,2.0
3,4.0,4.0,24.0,3.0
4,5.0,5.0,458.0,7.0
...,...,...,...,...
165,166.0,166.0,50.0,6.0
166,167.0,167.0,1046.0,2.0
167,168.0,168.0,2558.0,7.0
168,169.0,169.0,2740.0,9.0


In [756]:
parts_purchase_pd.to_csv("outputs/zakup_części.csv", index=False)

In [757]:
df_equipment = df_equipment[['id', 'nazwa', 'atrybut']]

In [758]:
df_equipment.to_csv("outputs/wyposażenie.csv", index=False)